# Temperature Data Extraction Tutorial

In [12]:
from matplotlib import pyplot as plt
import matplotlib 
%matplotlib inline
import pandas as pd
import gdal
import os
import datetime
from datetime import timedelta
import numpy as np
import fiona
import shapely
from shapely import geometry
from os.path import dirname
import glob
import sys
import pickle
from functools import partial

parent_dir = dirname(dirname(os.getcwd()))
sys.path.append(os.path.join(parent_dir,'StreamflowTempModel','lib'))
sys.path.append(os.path.join(parent_dir,'StreamflowTempModel','4_temperature'))
from temperature import SimpleTemperature
import zonal_stats as zs
import meteolib as meteo
import evaplib as evap

In [13]:
try:
    basins = glob.glob(os.path.join(parent_dir,'raw_data','basins_poly','*.shp'))[0]
except RuntimeError:
    print 'Cannot find basins shapefile. Please make sure basins shapefile is located in \n the model directory under /raw_data/basins_poly'

In [14]:
fc = fiona.open(basins)
shapefile_record = fc.next()
pos_dict={}
for shapefile_record in fc:
    shape = shapely.geometry.asShape(shapefile_record['geometry'])
    long_point = shape.centroid.coords.xy[0][0]
    lat_point = shape.centroid.coords.xy[1][0]
    pos = (long_point, lat_point)
    pos_dict[int(shapefile_record['properties']['cat'])]=pos
    

In [4]:
rew_config = pickle.load( open( os.path.join(parent_dir,'model_data','rew_config.p'), "rb" ) )
climate_group_forcing = pickle.load( open( os.path.join(parent_dir,'model_data','climate_group_forcing.p'), "rb" ) )
model_config = pickle.load( open( os.path.join(parent_dir, 'model_data', 'model_config.p'), 'rb'))
temperature_params = pickle.load( open( os.path.join(parent_dir, 'model_data', 'temperature_params.p'), 'rb'))

#start/stop dates for running model  
#spinup date is the date after start_date for which we assume model is finished spinning up         
start_date = model_config['start_date']
stop_date = model_config['stop_date']
spinup_date = model_config['spinup_date']
Tmax = model_config['Tmax']
dt = model_config['dt_channel']
t = model_config['t_channel']
resample_freq_channel = model_config['resample_freq_channel']
resample_freq_hillslope = model_config['resample_freq_hillslope']
timestamps_hillslope = model_config['timestamps_hillslope']
timestamps_channel = model_config['timestamps_channel']

In [ ]:
#Get daily min max 

for rew_id in rew_config.keys():
    climate_group = rew_config[rew_id]['climate_group']
    rew_df = climate_group_forcing[climate_group]
    start_date = model_config['start_date']
    stop_date = model_config['stop_date']
    temperature_forcing_df = pd.DataFrame([],index=model_config['timestamps_temperature'])
    
    #Calculate longwave incoming radiation using Konzelmann et al. 1994
    #Need temperature and vapor pressure 
    #interpolate these to timestamps_temperature resolution
    rng = pd.date_range(start_date,datetime.datetime(stop_date.year,stop_date.month, stop_date.day, 12, 0),freq='12H')
    interleaved = [val for pair in zip(rew_df['tmin'][start_date:stop_date], rew_df['tmax'][start_date:stop_date]) for val in pair]
    minmax = pd.DataFrame(interleaved, index=rng, columns=['ta'])
    empty = pd.DataFrame([], index=model_config['timestamps_temperature'])
    temp_ea = pd.concat([minmax, empty, rew_df['ea']],axis=1).interpolate(method='linear')
    sigma = 5.67e-8 #watts/(m^2 K^4)
    F = 1 #cloud fraction, here taken to be 1
    epscs = 0.23 + 0.4393*(1000*temp_ea['ea']/(temp_ea['ta']+273.16))**(1/7.0)
    temperature_forcing_df['ta'] = temp_ea['ta']
    temperature_forcing_df['Lin'] = epscs*F*sigma*(temp_ea['ta']+273.16)**4
    
    
    #Get shortwave radiation. Here, we simply use data collected at 
    #the Eel River Critical Zone Observatory. For the time being, 
    #this does not account for topographic/vegetation shading. Soon will use 
    #LiDAR light penetration index developed by Bode et. al. 2015
    shortwave = pickle.load( open( os.path.join(parent_dir, 'raw_data','shortwave','shortwave.p'), 'rb'))
    temperature_forcing_df = pd.concat([temperature_forcing_df, shortwave],axis=1).interpolate(method='linear')
    temperature_forcing_df = temperature_forcing_df.loc[(temperature_forcing_df.index>=model_config['timestamps_hillslope'][0]) & (temperature_forcing_df.index<= model_config['timestamps_hillslope'][-1])]
    
    